<a href="https://colab.research.google.com/github/KAFKA2306/hitaiall/blob/main/Hitaiall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from io import StringIO
import re
from datetime import datetime
#from config import Config
import os
import shutil

# config.pyファイルを作成
class Config:
    # Googleスプレッドシートの設定
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/136-Sc3wksQAjMw6FIqOw09zVdiIRRy-ewBShDUr-L5E/edit?gid=1869191766#gid=1869191766"
    spreadsheet_id = "136-Sc3wksQAjMw6FIqOw09zVdiIRRy-ewBShDUr-L5E"

    # キャッシュファイルのパス
    CACHE_FILE = "booth_item_cache.json"

    # GitHub設定 - 最小限の権限を持つトークンを使用
    github_token = "最小限の権限を持つトークンを使用"
    github_username = "KAFKA2306"
    github_repo = "hitaiall"
    github_email = "kafkafinancialgroup@gmail.com"

    # 出力ファイル名
    output_html = "index.html"

In [ ]:

def read_spreadsheet():
    spreadsheet_id = Config.spreadsheet_id

    # シート指定を含めたエクスポートURLを作成
    export_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?format=csv&gid=1869191766"

    try:
        # スプレッドシートデータを取得
        response = requests.get(export_url)
        # 文字化け対策：エンコーディングを明示的に設定
        response.encoding = 'utf-8'
        response.raise_for_status()

        # CSVデータをPandasデータフレームに変換（エンコーディング指定）
        raw_datasheet_df = pd.read_csv(
            StringIO(response.text),
            encoding='utf-8',
            dtype=str  # すべての列を文字列として読み込み
        )

        return raw_datasheet_df

    except requests.exceptions.RequestException as e:
        print(f"スプレッドシートの取得中にエラーが発生しました: {e}")
        return None

def extract_item_id(url):
    """URLからitem_idを抽出する関数"""
    if pd.isna(url):
        return None

    # 通常のBooth URL形式
    match = re.search(r'items/(\d+)', url)
    if match:
        return match.group(1)
    return None

def extract_twitter_id(twitter_text):
    """TwitterのIDを抽出する関数"""
    if pd.isna(twitter_text):
        return None

    # 文字列全体から不要な文字を削除して整理
    twitter_text = twitter_text.strip()

    # @から始まるパターン
    if twitter_text.startswith('@'):
        return twitter_text[1:]

    # x.com/{username}形式 - 波括弧付きのパターン
    match = re.search(r'x\.com/\{([^}]+)\}', twitter_text)
    if match:
        return match.group(1)

    # 通常のx.com/username形式
    match = re.search(r'x\.com/([^/?}\s]+)', twitter_text)
    if match:
        return match.group(1)

    # 単独の{username}形式
    match = re.search(r'\{([^}]+)\}', twitter_text)
    if match:
        return match.group(1)

    # 波括弧が閉じられていない場合
    match = re.search(r'\{([^{]+)', twitter_text)
    if match:
        return match.group(1)

    # ?や）などの特殊文字を削除
    cleaned_id = re.sub(r'[?）\(\)]', '', twitter_text)

    # その他の形式はそのまま返す
    return cleaned_id

def clean_dataframe(df):
    """データフレームをクリーンにして必要な情報を抽出する関数"""
    # 新しいデータフレームを作成
    clean_df = pd.DataFrame()

    # タイムスタンプを追加
    clean_df['timestamp'] = df.iloc[:, 0]

    # アバターのitem_idを抽出
    clean_df['avatar_item_id'] = df.iloc[:, 1].apply(extract_item_id)

    # 衣装のitem_idを抽出
    clean_df['costume_item_id'] = df.iloc[:, 2].apply(extract_item_id)

    # Twitter IDを抽出
    clean_df['twitter_id'] = df.iloc[:, 3].apply(extract_twitter_id)

    # 希望価格を抽出（数値に変換）
    clean_df['price'] = pd.to_numeric(df.iloc[:, 4], errors='coerce')

    return clean_df

# メイン処理
raw_datasheet_df = read_spreadsheet()
clean_datasheet_df = clean_dataframe(raw_datasheet_df)

In [ ]:
# 集計処理の共通関数
def aggregate_stats(df, group_by_columns):
    # グループ化して集計
    grouped = df.groupby(group_by_columns)

    # 集計結果を格納するデータフレーム
    result = pd.DataFrame({
        'count': grouped.size(),
        'total_price': grouped['price'].sum(),
        'avg_price': grouped['price'].mean(),
        'median_price': grouped['price'].median()
    })

    # median_priceが高い順番にソート
    result = result.sort_values(['count','median_price'], ascending=False)

    return result

# 各集計タイプの関数
def get_combination_data(df):
    """アバターIDと衣装IDの組み合わせごとの集計"""
    return aggregate_stats(df, ['avatar_item_id', 'costume_item_id'])

def get_avatar_ranking(df):
    """アバターIDごとの集計"""
    return aggregate_stats(df, 'avatar_item_id')

def get_costume_ranking(df):
    """衣装IDごとの集計"""
    return aggregate_stats(df, 'costume_item_id')

# メイン処理
aggregated_data = get_combination_data(clean_datasheet_df)
avatar_ranking_df = get_avatar_ranking(clean_datasheet_df)
costume_ranking_df = get_costume_ranking(clean_datasheet_df)

# 結果表示
print("アバターと衣装の組み合わせごとの集計 (件数順):")
display(aggregated_data)

print("\nアバターIDごとの集計 (件数順):")
display(avatar_ranking_df)

print("\n衣装IDごとの集計 (件数順):")
display(costume_ranking_df)

アバターと衣装の組み合わせごとの集計 (件数順):


,,count,total_price,avg_price,median_price
avatar_item_id,costume_item_id,,,,
6213757,4443215,1,7000.0,7000.0,7000.0
5331716,5118037,1,6000.0,6000.0,6000.0
6213757,4421581,1,6000.0,6000.0,6000.0
4930863,5955253,1,3500.0,3500.0,3500.0
5132797,6110958,1,3500.0,3500.0,3500.0
5832850,6447797,1,3000.0,3000.0,3000.0
6106863,3612336,1,3000.0,3000.0,3000.0
4655445,6299638,1,2500.0,2500.0,2500.0
5650156,5359699,1,2500.0,2500.0,2500.0



アバターIDごとの集計 (件数順):


,count,total_price,avg_price,median_price
avatar_item_id,,,,
6213757,2,13000.0,6500.0,6500.0
4825073,2,3500.0,1750.0,1750.0
5650156,2,3000.0,1500.0,1500.0
5331716,1,6000.0,6000.0,6000.0
4930863,1,3500.0,3500.0,3500.0
5132797,1,3500.0,3500.0,3500.0
5832850,1,3000.0,3000.0,3000.0
6106863,1,3000.0,3000.0,3000.0
4655445,1,2500.0,2500.0,2500.0



衣装IDごとの集計 (件数順):


,count,total_price,avg_price,median_price
costume_item_id,,,,
6110958,2,5000.0,2500.0,2500.0
4443215,1,7000.0,7000.0,7000.0
4421581,1,6000.0,6000.0,6000.0
5118037,1,6000.0,6000.0,6000.0
5955253,1,3500.0,3500.0,3500.0
3612336,1,3000.0,3000.0,3000.0
6447797,1,3000.0,3000.0,3000.0
5359699,1,2500.0,2500.0,2500.0
6299638,1,2500.0,2500.0,2500.0


In [ ]:
import pandas as pd
import json
import os
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import re

# キャッシュファイルのパス
CACHE_FILE = Config.CACHE_FILE

# キャッシュをファイルから読み込む
def load_cache():
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return {}

# キャッシュをファイルに保存
def save_cache(cache):
    with open(CACHE_FILE, "w", encoding="utf-8") as f:
        json.dump(cache, f, ensure_ascii=False, indent=4)

# 改善版Boothアイテム情報取得関数
def fetch_booth_item_info_improved(item_id, cache=None):
    # キャッシュが指定されていない場合は読み込む
    if cache is None:
        cache = load_cache()

    # 無効なitem_idの場合は空の情報を返す
    if not item_id:
        return {
            'item_name': None,
            'creator_id': None,
            'shop_name': None,
            'image_url': None,
            'price': None,
            'url': None
        }

    # キャッシュにある場合はキャッシュから返す
    if item_id in cache:
        #print(f"キャッシュヒット: item_id {item_id}")
        return cache[item_id]

    print(f"データ取得中: item_id {item_id}")
    url = f"https://booth.pm/ja/items/{item_id}"

    try:
        # リクエスト間隔を空ける（サーバー負荷軽減のため）
        time.sleep(1)

        # 商品ページを取得
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept-Language': 'ja,en-US;q=0.9,en;q=0.8'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # BeautifulSoupでHTMLを解析
        soup = BeautifulSoup(response.text, 'html.parser')

        # 商品名を取得（複数のセレクタを試す）
        item_name = "不明"
        selectors = [
            'h1.item-name',
            'h1.u-tpg-title1',
            'h1[itemprop="name"]',
            'meta[property="og:title"]'
        ]

        for selector in selectors:
            elem = soup.select_one(selector)
            if elem:
                if selector == 'meta[property="og:title"]' and 'content' in elem.attrs:
                    item_name = elem['content']
                else:
                    item_name = elem.text.strip()
                break

        # ショップ名を取得（複数のセレクタを試す）
        shop_name = "不明"
        shop_selectors = [
            'a.shop-name',
            'div.u-text-ellipsis > a',
            'a[itemprop="author"]',
            'meta[property="og:site_name"]'
        ]

        for selector in shop_selectors:
            elem = soup.select_one(selector)
            if elem:
                if selector == 'meta[property="og:site_name"]' and 'content' in elem.attrs:
                    shop_name = elem['content']
                else:
                    shop_name = elem.text.strip()
                break

        # 作者IDを取得（URLから）
        creator_id = None

        # ショップのURLからcreator_idを抽出
        for link in soup.find_all('a', href=True):
            href = link['href']
            # ショップページへのリンクを探す
            shop_match = re.search(r'https?://([^.]+)\.booth\.pm', href)
            if shop_match:
                creator_id = shop_match.group(1)
                break

        # 上記で見つからなければ、別のパターンを試す
        if not creator_id:
            for link in soup.find_all('a', href=True):
                href = link['href']
                if '/shop/' in href:
                    creator_id = href.split('/shop/')[-1].split('?')[0].split('#')[0]
                    break

        # それでも見つからなければ、ページURLから推測
        if not creator_id:
            url_match = re.search(r'https?://([^.]+)\.booth\.pm', response.url)
            if url_match:
                creator_id = url_match.group(1)

        # 商品画像URLを取得（複数のセレクタを試す）
        image_url = None
        image_selectors = [
            'img.market-item-detail-item-image',
            'img[itemprop="image"]',
            'meta[property="og:image"]'
        ]

        for selector in image_selectors:
            elem = soup.select_one(selector)
            if elem:
                if selector == 'meta[property="og:image"]' and 'content' in elem.attrs:
                    image_url = elem['content']
                elif 'src' in elem.attrs:
                    image_url = elem['src']
                break

        # 価格を取得（複数のセレクタを試す）
        price = None
        price_selectors = [
            'div.price',
            'span[itemprop="price"]',
            'meta[property="og:price:amount"]'
        ]

        for selector in price_selectors:
            elem = soup.select_one(selector)
            if elem:
                if selector == 'meta[property="og:price:amount"]' and 'content' in elem.attrs:
                    price = elem['content']
                else:
                    price_text = elem.text.strip()
                    price_match = re.search(r'¥\s*([\d,]+)', price_text)
                    if price_match:
                        price = price_match.group(1).replace(',', '')
                break

        item_data = {
            'item_name': item_name,
            'creator_id': creator_id,
            'shop_name': shop_name,
            'image_url': image_url,
            'price': price,
            'url': url
        }

        # キャッシュを更新
        cache[item_id] = item_data
        save_cache(cache)

        return item_data

    except Exception as e:
        print(f"商品ID {item_id} の情報取得中にエラーが発生しました: {e}")
        error_data = {
            'item_name': f"エラー: {str(e)[:50]}...",
            'creator_id': None,
            'shop_name': None,
            'image_url': None,
            'price': None,
            'url': url
        }

        # エラー情報もキャッシュに保存（再試行を減らすため）
        cache[item_id] = error_data
        save_cache(cache)

        return error_data

# アバターIDと衣装IDの情報を取得して整理する関数
def rankings_with_item_info_improved(avatar_ranking_df, costume_ranking_df):
    # キャッシュを読み込む
    cache = load_cache()

    # アバターIDごとの情報を取得
    print("アバター情報を取得中...")
    avatar_info_list = []
    for idx, row in avatar_ranking_df.reset_index().iterrows():
        avatar_id = row['avatar_item_id']
        item_info = fetch_booth_item_info_improved(avatar_id, cache)

        # 集計情報と商品情報を結合
        combined_info = {
            'avatar_item_id': avatar_id,
            'count': row.get('count', None),
            'total_price': row.get('total_price', None),
            'avg_price': row.get('avg_price', None),
            'median_price': row.get('median_price', None),
            'item_name': item_info['item_name'],
            'creator_id': item_info['creator_id'],
            'shop_name': item_info['shop_name'],
            'booth_price': item_info['price'],
            'image_url': item_info['image_url'],
            'url': item_info['url']
        }
        avatar_info_list.append(combined_info)

    # 衣装IDごとの情報を取得
    print("\n衣装情報を取得中...")
    costume_info_list = []
    for idx, row in costume_ranking_df.reset_index().iterrows():
        costume_id = row['costume_item_id']
        if pd.isna(costume_id):
            continue
        item_info = fetch_booth_item_info_improved(costume_id, cache)

        # 集計情報と商品情報を結合
        combined_info = {
            'costume_item_id': costume_id,
            'count': row.get('count', None),
            'total_price': row.get('total_price', None),
            'avg_price': row.get('avg_price', None),
            'median_price': row.get('median_price', None),
            'item_name': item_info['item_name'],
            'creator_id': item_info['creator_id'],
            'shop_name': item_info['shop_name'],
            'booth_price': item_info['price'],
            'image_url': item_info['image_url'],
            'url': item_info['url']
        }
        costume_info_list.append(combined_info)

    # データフレームに変換
    avatar_df = pd.DataFrame(avatar_info_list)
    costume_df = pd.DataFrame(costume_info_list)

    return avatar_df, costume_df

# メイン処理
avatar_df, costume_df = rankings_with_item_info_improved(avatar_ranking_df, costume_ranking_df)

# 結果表示
print("\n=== アバターIDごとの詳細情報（改善版） ===")
display(avatar_df)

print("\n=== 衣装IDごとの詳細情報（改善版） ===")
display(costume_df)


アバター情報を取得中...

衣装情報を取得中...

=== アバターIDごとの詳細情報（改善版） ===


,avatar_item_id,count,total_price,avg_price,median_price,item_name,creator_id,shop_name,booth_price,image_url,url
0,6213757,2,13000.0,6500.0,6500.0,オリジナル3Dモデル - ポワン #Powan3D - Senna Studio - BOOTH,senna-studio,Senna Studio,1500,https://booth.pximg.net/70c03f34-f561-4b7a-af1...,https://booth.pm/ja/items/6213757
1,4825073,2,3500.0,1750.0,1750.0,【オリジナル3Dモデル】Lasyusha〈ラシューシャ〉 - Keenoo - BOOTH,keenooshop,Keenoo,1800,https://booth.pximg.net/de9ac79a-d2ba-4cc5-a4c...,https://booth.pm/ja/items/4825073
2,5650156,2,3000.0,1500.0,1500.0,【オリジナル3Dモデル】 Sio / しお / ver.2.00 - Chocolate r...,chocolaterice,Chocolate rice,1700,https://booth.pximg.net/817e9a9a-020c-4fac-8b2...,https://booth.pm/ja/items/5650156
3,5331716,1,6000.0,6000.0,6000.0,オリジナル3Dモデル『銀杏』 - みらいショップ - BOOTH,mtshop,みらいショップ,0,https://booth.pximg.net/922e944c-e396-411f-bda...,https://booth.pm/ja/items/5331716
4,4930863,1,3500.0,3500.0,3500.0,【3Dmodel】悪魔リーマン【VRChat avatar】 - kaibatu2mm LA...,kaibatu2mm,kaibatu2mm LAB,20000,https://booth.pximg.net/b1b300fb-f459-487f-8fd...,https://booth.pm/ja/items/4930863
5,5132797,1,3500.0,3500.0,3500.0,オリジナル3Dモデル「瑞希」メニューギミック搭載 - IKUSIA - BOOTH,paryi,IKUSIA,5000,https://booth.pximg.net/96d1d589-6879-4d30-889...,https://booth.pm/ja/items/5132797
6,5832850,1,3000.0,3000.0,3000.0,【リーサルフリート】オリジナル3Dモデル - VERMILION .Studio - BOOTH,kv2,VERMILION .Studio,1000,https://booth.pximg.net/e6deeb51-72a5-4562-942...,https://booth.pm/ja/items/5832850
7,6106863,1,3000.0,3000.0,3000.0,オリジナル3Dモデル「しなの」 - ポンデロニウム研究所 - BOOTH,ponderogen,ポンデロニウム研究所,3000,https://booth.pximg.net/ed52788c-0b3b-4e38-9de...,https://booth.pm/ja/items/6106863
8,4655445,1,2500.0,2500.0,2500.0,【ラムダ】オリジナル3Dモデル - VERMILION .Studio - BOOTH,kv2,VERMILION .Studio,6000,https://booth.pximg.net/e6deeb51-72a5-4562-942...,https://booth.pm/ja/items/4655445
9,3923094,1,2000.0,2000.0,2000.0,【オリジナル３Dモデル】龍のヨルちゃん - KUYUYU/電脳屋 - BOOTH,skd-noratama,KUYUYU/電脳屋,6000,https://booth.pximg.net/3923a6a5-7600-4cc2-924...,https://booth.pm/ja/items/3923094



=== 衣装IDごとの詳細情報（改善版） ===


,costume_item_id,count,total_price,avg_price,median_price,item_name,creator_id,shop_name,booth_price,image_url,url
0,6110958,2,5000.0,2500.0,2500.0,🩷本命ニット🩷【15アバター対応】 - てんぱすおおもり - BOOTH,tempasta,てんぱすおおもり,500,https://booth.pximg.net/2bbfc6cd-c88d-4f80-8d4...,https://booth.pm/ja/items/6110958
1,4443215,1,7000.0,7000.0,7000.0,レイヤードパーカー（LAYERD PARKA） - P_Store - BOOTH,poppo-shop,P_Store,500,https://booth.pximg.net/fd080b93-50c9-4a89-a7f...,https://booth.pm/ja/items/4443215
2,4421581,1,6000.0,6000.0,6000.0,モノクロゴースト - P_Store - BOOTH,poppo-shop,P_Store,500,https://booth.pximg.net/fd080b93-50c9-4a89-a7f...,https://booth.pm/ja/items/4421581
3,5118037,1,6000.0,6000.0,6000.0,【VRC想定衣装】Memoriabelle - ふぇざーしーぷ - BOOTH,feathersheep,ふぇざーしーぷ,1500,https://booth.pximg.net/66f34f45-d82c-4379-baf...,https://booth.pm/ja/items/5118037
4,5955253,1,3500.0,3500.0,3500.0,【3Dモデル】ANUBIS (+HEAD Update!) - DIMGRAY - BOOTH,dimgray,DIMGRAY,1800,https://booth.pximg.net/a47c4793-3a41-4936-a02...,https://booth.pm/ja/items/5955253
5,3612336,1,3000.0,3000.0,3000.0,【テレクレア用衣装】チャイナドレス - SELECT SHOP -Cornet- - BOOTH,selectshop,SELECT SHOP -Cornet-,1700,https://booth.pximg.net/5494d858-22e9-43e3-87a...,https://booth.pm/ja/items/3612336
6,6447797,1,3000.0,3000.0,3000.0,[16Avatars]𝑵𝒐𝒊𝒓_𝑳𝒖𝒙𝒆 - velvetsky - BOOTH,velvetsky,velvetsky,0,https://booth.pximg.net/9cef401b-c87a-471e-a3b...,https://booth.pm/ja/items/6447797
7,5359699,1,2500.0,2500.0,2500.0,【12アバター対応】ナイト・イン・ヨシワラ【MA対応】 - VAGRANT - BOOTH,vagrant,VAGRANT,2200,https://booth.pximg.net/4a8b2e27-d374-4781-866...,https://booth.pm/ja/items/5359699
8,6299638,1,2500.0,2500.0,2500.0,『シック・グレイス』 ‐ Chic Grace【11アバター対応】 - Délice Hau...,delicehaute,Délice Haute,2000,https://booth.pximg.net/943f601f-9c03-4761-be7...,https://booth.pm/ja/items/6299638
9,5809341,1,2000.0,2000.0,2000.0,【9アバター対応】 Cross Maid ✖ GLAY Unknown - Chocolat...,chocolaterice,Chocolate rice,1700,https://booth.pximg.net/817e9a9a-020c-4fac-8b2...,https://booth.pm/ja/items/5809341


In [ ]:
# メイン処理
raw_datasheet_df = read_spreadsheet()
clean_datasheet_df = clean_dataframe(raw_datasheet_df)

# メイン処理
combination_data = get_combination_data(clean_datasheet_df)
avatar_ranking_df = get_avatar_ranking(clean_datasheet_df)
costume_ranking_df = get_costume_ranking(clean_datasheet_df)

# メイン処理
avatar_df, costume_df = rankings_with_item_info_improved(avatar_ranking_df, costume_ranking_df)


# ダッシュボード準備
# 上位5件のデータを抽出
combination_top5 = combination_data.reset_index()
avatar_top5 = avatar_ranking_df.head(5)
costume_top5 = costume_ranking_df.head(5)

# 組み合わせデータに名前と画像を追加
combinations = pd.merge(
    combination_top5,
    avatar_df[['avatar_item_id', 'item_name', 'shop_name', 'image_url', 'url']],
    on='avatar_item_id',
    how='left'
)
combinations = pd.merge(
    combinations,
    costume_df[['costume_item_id', 'item_name', 'shop_name', 'image_url', 'url']],
    on='costume_item_id',
    how='left',
    suffixes=('_avatar', '_costume')
)

# アバターランキングに名前と画像を追加
avatars = pd.merge(
    avatar_top5,
    avatar_df[['avatar_item_id', 'item_name', 'shop_name', 'image_url', 'url']],
    on='avatar_item_id',
    how='left'
)

# 衣装ランキングに名前と画像を追加
costumes = pd.merge(
    costume_top5,
    costume_df[['costume_item_id', 'item_name', 'shop_name', 'image_url', 'url']],
    on='costume_item_id',
    how='left'
)

アバター情報を取得中...

衣装情報を取得中...


In [ ]:
import pandas as pd
from IPython.display import display, HTML

class DashboardConfig:
    # 色設定 - 視認性向上版
    COLORS = {
        'primary': '#2980b9',         # より濃い青 - コントラスト向上
        'secondary': '#34495e',       # より濃い紺色 - 読みやすさ向上
        'background': '#f9f9f9',      # わずかに灰色がかった背景 - 目の疲れ軽減
        'text': '#2c3e50',            # 濃い青灰色 - 黒よりも目に優しい
        'light_text': '#5d6d7e',      # より濃いグレー - 薄すぎず読みやすい
        'border': '#bdc3c7',          # より濃い境界線 - 視認性向上
        'hover': '#d6eaf8',           # より濃いホバー色 - 明確な状態変化
        'odd_row': '#ecf0f1',         # より明確な行の区別
        'even_row': '#f5f7f8',        # 白に近いが完全な白ではない
        'header_bg': '#2471a3',       # ヘッダー背景 - アクセシビリティ向上
        'header_text': '#ffffff'      # ヘッダーテキスト - 高コントラスト
    }

    # フォント設定 - 日本語対応
    FONTS = {
        'main': '"Segoe UI", Meiryo, "Hiragino Sans", "Hiragino Kaku Gothic ProN", sans-serif',  # 日本語フォントを優先
        'monospace': '"Consolas", "Courier New", "MS Gothic", monospace'  # 日本語等幅フォント対応
    }

    # サイズ設定
    SIZES = {
        'image_width': 120,
        'min_cell_width': 200,
        'rank_badge_size': 30,
        'section_margin': 40,
        'border_radius': 8
    }

    # 表示する列の設定
    COLUMNS = {
        'combination': [
            {'id': 'rank', 'name': '順位', 'type': 'rank'},
            {'id': 'avatar', 'name': 'アバター', 'type': 'item'},
            {'id': 'costume', 'name': '衣装', 'type': 'item'},
            {'id': 'count', 'name': '件数', 'type': 'price'},
            {'id': 'total_price', 'name': '合計金額', 'type': 'price', 'suffix': '円'},
            {'id': 'avg_price', 'name': '平均価格', 'type': 'price', 'suffix': '円'},
            {'id': 'median_price', 'name': '中央値価格', 'type': 'price', 'suffix': '円'}
        ],
        'avatar': [
            {'id': 'rank', 'name': '順位', 'type': 'rank'},
            {'id': 'avatar', 'name': 'アバター', 'type': 'item'},
            {'id': 'count', 'name': '件数', 'type': 'price'},
            {'id': 'total_price', 'name': '合計金額', 'type': 'price', 'suffix': '円'},
            {'id': 'avg_price', 'name': '平均価格', 'type': 'price', 'suffix': '円'},
            {'id': 'median_price', 'name': '中央値価格', 'type': 'price', 'suffix': '円'}
        ],
        'costume': [
            {'id': 'rank', 'name': '順位', 'type': 'rank'},
            {'id': 'costume', 'name': '衣装', 'type': 'item'},
            {'id': 'count', 'name': '件数', 'type': 'price'},
            {'id': 'total_price', 'name': '合計金額', 'type': 'price', 'suffix': '円'},
            {'id': 'avg_price', 'name': '平均価格', 'type': 'price', 'suffix': '円'},
            {'id': 'median_price', 'name': '中央値価格', 'type': 'price', 'suffix': '円'}
        ]
    }

    # セクション設定
    SECTIONS = [
        {'id': 'combination', 'title': '人気アバター＆衣装の組み合わせランキング'},
        {'id': 'avatar', 'title': '人気アバターランキング'},
        {'id': 'costume', 'title': '人気衣装ランキング'}
    ]

    @classmethod
    def get_css(cls):
        """CSSスタイルを生成"""
        return f"""
        <style>
            .dashboard {{
                font-family: {cls.FONTS['main']};
                margin: 20px;
                color: {cls.COLORS['text']};
            }}
            .section {{
                margin-bottom: {cls.SIZES['section_margin']}px;
                background-color: {cls.COLORS['background']};
                border-radius: {cls.SIZES['border_radius']}px;
                box-shadow: 0 2px 5px rgba(0,0,0,0.1);
                padding: 20px;
            }}
            .section h2 {{
                color: {cls.COLORS['secondary']};
                border-bottom: 2px solid {cls.COLORS['primary']};
                padding-bottom: 10px;
                margin-top: 0;
            }}
            .ranking-table {{
                border-collapse: collapse;
                width: 100%;
                margin-top: 15px;
                box-shadow: 0 1px 3px rgba(0,0,0,0.1);
            }}
            .ranking-table th, .ranking-table td {{
                border: 1px solid {cls.COLORS['border']};
                padding: 12px;
                text-align: left;
            }}
            .ranking-table th {{
                background-color: {cls.COLORS['primary']};
                color: white;
                font-weight: bold;
            }}
            .ranking-table tr:nth-child(odd) {{
                background-color: {cls.COLORS['odd_row']};
            }}
            .ranking-table tr:nth-child(even) {{
                background-color: {cls.COLORS['even_row']};
            }}
            .ranking-table tr:hover {{
                background-color: {cls.COLORS['hover']};
            }}
            .item-image {{
                width: {cls.SIZES['image_width']}px;
                height: auto;
                border-radius: 6px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                transition: transform 0.2s;
                display: block;
                margin-bottom: 8px;
            }}
            .item-image:hover {{
                transform: scale(1.05);
            }}
            .item-name {{
                font-weight: bold;
                color: {cls.COLORS['primary']};
                text-decoration: none;
                display: block;
                margin-bottom: 4px;
            }}
            .item-name:hover {{
                text-decoration: underline;
            }}
            .shop-name {{
                color: {cls.COLORS['light_text']};
                font-size: 0.9em;
                display: block;
            }}
            .price-data {{
                text-align: right;
                font-family: {cls.FONTS['monospace']};
                color: {cls.COLORS['secondary']};
            }}
            .rank-number {{
                font-weight: bold;
                text-align: center;
                background-color: {cls.COLORS['primary']};
                color: white;
                border-radius: 50%;
                width: {cls.SIZES['rank_badge_size']}px;
                height: {cls.SIZES['rank_badge_size']}px;
                display: flex;
                align-items: center;
                justify-content: center;
                margin: 0 auto;
            }}
            .item-cell {{
                min-width: {cls.SIZES['min_cell_width']}px;
            }}
        </style>
        """

class DashboardRenderer:
    @staticmethod
    def render_table_header(columns):
        """テーブルヘッダーを生成"""
        header = "<tr>"
        for column in columns:
            header += f"<th>{column['name']}</th>"
        header += "</tr>"
        return header

    @staticmethod
    def render_rank_cell(rank):
        """順位セルを生成"""
        return f'<td><div class="rank-number">{rank}</div></td>'

    @staticmethod
    def render_item_cell(item):
        """アイテムセルを生成"""
        return f"""
        <td class="item-cell">
            <img src="{item['image_url']}" class="item-image" onerror="this.src='https://via.placeholder.com/120x120?text=No+Image'">
            <a href="{item['url']}" class="item-name" target="_blank">{item['item_name']}</a>
            <span class="shop-name">{item['shop_name']}</span>
        </td>
        """

    @staticmethod
    def render_price_cell(value, suffix=""):
        """価格セルを生成"""
        if pd.isna(value):
            formatted_value = "N/A"
        elif isinstance(value, (int, float)):
            formatted_value = f"{int(value):,}{suffix}"
        else:
            formatted_value = f"{value}{suffix}"
        return f'<td class="price-data">{formatted_value}</td>'

    @classmethod
    def render_combination_row(cls, i, row):
        """組み合わせ行を生成"""
        html = "<tr>"
        html += cls.render_rank_cell(i+1)

        # アバターセル
        avatar_item = {
            'image_url': row['image_url_avatar'],
            'url': row['url_avatar'],
            'item_name': row['item_name_avatar'],
            'shop_name': row['shop_name_avatar']
        }
        html += cls.render_item_cell(avatar_item)

        # 衣装セル
        costume_item = {
            'image_url': row['image_url_costume'],
            'url': row['url_costume'],
            'item_name': row['item_name_costume'],
            'shop_name': row['shop_name_costume']
        }
        html += cls.render_item_cell(costume_item)

        # 統計データセル
        html += cls.render_price_cell(row['count'])
        html += cls.render_price_cell(row['total_price'], '円')
        html += cls.render_price_cell(row['avg_price'], '円')
        html += cls.render_price_cell(row['median_price'], '円')

        html += "</tr>"
        return html

    @classmethod
    def render_avatar_row(cls, i, row):
        """アバター行を生成"""
        html = "<tr>"
        html += cls.render_rank_cell(i+1)

        # アバターセル
        avatar_item = {
            'image_url': row['image_url'],
            'url': row['url'],
            'item_name': row['item_name'],
            'shop_name': row['shop_name']
        }
        html += cls.render_item_cell(avatar_item)

        # 統計データセル
        html += cls.render_price_cell(row['count'])
        html += cls.render_price_cell(row['total_price'], '円')
        html += cls.render_price_cell(row['avg_price'], '円')
        html += cls.render_price_cell(row['median_price'], '円')

        html += "</tr>"
        return html

    @classmethod
    def render_costume_row(cls, i, row):
        """衣装行を生成"""
        html = "<tr>"
        html += cls.render_rank_cell(i+1)

        # 衣装セル
        costume_item = {
            'image_url': row['image_url'],
            'url': row['url'],
            'item_name': row['item_name'],
            'shop_name': row['shop_name']
        }
        html += cls.render_item_cell(costume_item)

        # 統計データセル
        html += cls.render_price_cell(row['count'])
        html += cls.render_price_cell(row['total_price'], '円')
        html += cls.render_price_cell(row['avg_price'], '円')
        html += cls.render_price_cell(row['median_price'], '円')

        html += "</tr>"
        return html

    @classmethod
    def render_section(cls, section_id, title, data, render_row_func):
        """セクションを生成"""
        html = f"""
        <div class="section">
            <h2>{title}</h2>
            <table class="ranking-table">
                {cls.render_table_header(DashboardConfig.COLUMNS[section_id])}
        """

        for i, row in enumerate(data.iterrows()):
            html += render_row_func(i, row[1])

        html += """
            </table>
        </div>
        """
        return html

    @classmethod
    def render_dashboard(cls, combinations, avatars, costumes):
        """ダッシュボード全体を生成"""
        html = DashboardConfig.get_css()
        html += '<div class="dashboard">'

        # 組み合わせセクション
        html += cls.render_section(
            'combination',
            '人気アバター＆衣装の組み合わせランキング',
            combinations,
            cls.render_combination_row
        )

        # アバターセクション
        html += cls.render_section(
            'avatar',
            '人気アバターランキング',
            avatars,
            cls.render_avatar_row
        )

        # 衣装セクション
        html += cls.render_section(
            'costume',
            '人気衣装ランキング',
            costumes,
            cls.render_costume_row
        )

        html += '</div>'
        return html

In [ ]:
def create_dashboard(combinations, avatars, costumes):
    """ダッシュボードを作成して表示"""
    html_output = DashboardRenderer.render_dashboard(
        combinations,
        avatars,
        costumes
    )
    return html_output

# メイン処理
dashboard_html = create_dashboard(combinations, avatars, costumes)

# HTMLをファイルに保存
def save_dashboard_to_file(html_content, filename="index.html"):
    with open(filename, "w", encoding="utf-8") as f:
        # 完全なHTMLドキュメントとして保存
        complete_html = f"""<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>アバター＆衣装需要ダッシュボード</title>
</head>
<body>
    {html_content}
</body>
</html>"""
        f.write(complete_html)
    print(f"ダッシュボードを {filename} に保存しました")

# ダッシュボードをファイルに保存
save_dashboard_to_file(dashboard_html)

# ダッシュボードを表示
display(HTML(dashboard_html))


ダッシュボードを index.html に保存しました


順位,アバター,衣装,件数,合計金額,平均価格,中央値価格
1,オリジナル3Dモデル - ポワン #Powan3D - Senna Studio - BOOTH Senna Studio,レイヤードパーカー（LAYERD PARKA） - P_Store - BOOTH P_Store,1,"7,000円","7,000円","7,000円"
2,オリジナル3Dモデル『銀杏』 - みらいショップ - BOOTH みらいショップ,【VRC想定衣装】Memoriabelle - ふぇざーしーぷ - BOOTH ふぇざーしーぷ,1,"6,000円","6,000円","6,000円"
3,オリジナル3Dモデル - ポワン #Powan3D - Senna Studio - BOOTH Senna Studio,モノクロゴースト - P_Store - BOOTH P_Store,1,"6,000円","6,000円","6,000円"
4,【3Dmodel】悪魔リーマン【VRChat avatar】 - kaibatu2mm LAB - BOOTH kaibatu2mm LAB,【3Dモデル】ANUBIS (+HEAD Update!) - DIMGRAY - BOOTH DIMGRAY,1,"3,500円","3,500円","3,500円"
5,オリジナル3Dモデル「瑞希」メニューギミック搭載 - IKUSIA - BOOTH IKUSIA,🩷本命ニット🩷【15アバター対応】 - てんぱすおおもり - BOOTH てんぱすおおもり,1,"3,500円","3,500円","3,500円"
6,【リーサルフリート】オリジナル3Dモデル - VERMILION .Studio - BOOTH VERMILION .Studio,[16Avatars]𝑵𝒐𝒊𝒓_𝑳𝒖𝒙𝒆 - velvetsky - BOOTH velvetsky,1,"3,000円","3,000円","3,000円"
7,オリジナル3Dモデル「しなの」 - ポンデロニウム研究所 - BOOTH ポンデロニウム研究所,【テレクレア用衣装】チャイナドレス - SELECT SHOP -Cornet- - BOOTH SELECT SHOP -Cornet-,1,"3,000円","3,000円","3,000円"
8,【ラムダ】オリジナル3Dモデル - VERMILION .Studio - BOOTH VERMILION .Studio,『シック・グレイス』 ‐ Chic Grace【11アバター対応】 - Délice Haute - BOOTH Délice Haute,1,"2,500円","2,500円","2,500円"
9,【オリジナル3Dモデル】 Sio / しお / ver.2.00 - Chocolate rice - BOOTH Chocolate rice,【12アバター対応】ナイト・イン・ヨシワラ【MA対応】 - VAGRANT - BOOTH VAGRANT,1,"2,500円","2,500円","2,500円"
10,【オリジナル３Dモデル】龍のヨルちゃん - KUYUYU/電脳屋 - BOOTH KUYUYU/電脳屋,【9アバター対応】 Cross Maid ✖ GLAY Unknown - Chocolate rice - BOOTH Chocolate rice,1,"2,000円","2,000円","2,000円"


In [ ]:
def upload_to_github(filename=Config.output_html):
    """
    生成したHTMLファイルをGitHubリポジトリにアップロードする関数
    詳細なデバッグ情報を出力するように改良
    """
    # configから設定を読み込み
    token = Config.github_token
    user_name = Config.github_username
    repo_name = Config.github_repo
    email = Config.github_email

    print("=== 処理開始: GitHubアップロード ===")
    print(f"設定情報: ユーザー名={user_name}, リポジトリ名={repo_name}")
    print(f"アップロードファイル: {filename}")

    # トークンが空でないか確認
    if not token:
        print("エラー: GitHub トークンが設定されていません。")
        return

    # ファイルが存在するか確認
    if not os.path.exists(filename):
        print(f"エラー: アップロードするファイル {filename} が見つかりません。")
        return

    try:
        print(f"ファイルサイズ: {os.path.getsize(filename)} バイト")
        with open(filename, 'r', encoding='utf-8') as f:
            file_preview = f.read(100)  # 最初の100文字だけ表示
        print(f"ファイル内容プレビュー: {file_preview}...")
    except Exception as e:
        print(f"ファイル読み込み中にエラー: {str(e)}")

    print(f"\n--- GitHubリポジトリ {user_name}/{repo_name} にアップロード開始 ---")

    try:
        # 現在の作業ディレクトリを表示
        print(f"現在の作業ディレクトリ: {os.getcwd()}")

        # Gitの設定
        print("\n1. Gitの設定を行います...")
        os.system(f'git config --global user.email "{email}"')
        os.system(f'git config --global user.name "{user_name}"')
        print("Git設定完了")

        # リポジトリをクローン
        clone_path = f"/content/{repo_name}"
        print(f"\n2. リポジトリをクローンします: {clone_path}")

        # すでにクローンされている場合は削除
        if os.path.exists(clone_path):
            print(f"既存のクローンを削除: {clone_path}")
            os.system(f'rm -rf {clone_path}')

        # リポジトリをクローン
        clone_command = f'git clone https://{token}@github.com/{user_name}/{repo_name}.git {clone_path}'
        # トークンを隠した形でコマンドを表示
        print(f"実行コマンド: git clone https://***@github.com/{user_name}/{repo_name}.git {clone_path}")
        clone_result = os.system(clone_command)

        if clone_result != 0:
            print(f"エラー: リポジトリのクローンに失敗しました。終了コード: {clone_result}")
            return

        print("リポジトリのクローン成功")

        # クローンされたディレクトリが存在するか確認
        if not os.path.exists(clone_path):
            print(f"エラー: クローン後のディレクトリ {clone_path} が見つかりません。")
            return

        # ディレクトリ内容を確認
        print(f"\n3. クローンされたリポジトリの内容を確認:")
        os.system(f'ls -la {clone_path}')

        # 生成したファイルをリポジトリにコピー
        print(f"\n4. ファイルをリポジトリにコピーします:")
        dest_path = os.path.join(clone_path, filename)
        print(f"コピー元: {filename}")
        print(f"コピー先: {dest_path}")

        try:
            shutil.copy2(filename, dest_path)
            print("ファイルコピー成功")
        except Exception as e:
            print(f"ファイルコピー中にエラー: {str(e)}")
            return

        # コピー後のファイルが存在するか確認
        if not os.path.exists(dest_path):
            print(f"エラー: コピー先 {dest_path} にファイルが見つかりません。")
            return

        print(f"コピー後のファイルサイズ: {os.path.getsize(dest_path)} バイト")

        # GitHubにプッシュ
        print(f"\n5. GitHubにプッシュします...")

        # 現在のディレクトリを変更
        current_dir = os.getcwd()
        print(f"現在のディレクトリ: {current_dir}")
        print(f"移動先ディレクトリ: {clone_path}")
        os.chdir(clone_path)
        print(f"移動後のディレクトリ: {os.getcwd()}")

        # 変更を確認
        print("\n6. 変更を確認します (git status):")
        os.system('git status')

        # 強制的に追加
        print(f"\n7. ファイルを追加します (git add):")
        add_result = os.system(f'git add -f {filename}')
        if add_result != 0:
            print(f"エラー: ファイルの追加に失敗しました。終了コード: {add_result}")
            os.chdir(current_dir)  # 元のディレクトリに戻る
            return

        # 変更を再確認
        print("\n8. 追加後の状態を確認します (git status):")
        os.system('git status')

        # コミット
        print("\n9. 変更をコミットします (git commit):")
        commit_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        commit_message = f"ダッシュボードを更新 ({commit_date})"
        commit_result = os.system(f'git commit -m "{commit_message}"')
        if commit_result != 0:
            print(f"エラー: コミットに失敗しました。終了コード: {commit_result}")
            os.chdir(current_dir)  # 元のディレクトリに戻る
            return

        # プッシュ
        print("\n10. 変更をプッシュします (git push):")
        push_command = f'git push origin main'
        push_result = os.system(push_command)
        if push_result != 0:
            print(f"エラー: プッシュに失敗しました。終了コード: {push_result}")
            os.chdir(current_dir)  # 元のディレクトリに戻る
            return

        # 元のディレクトリに戻る
        os.chdir(current_dir)
        print(f"元のディレクトリに戻りました: {os.getcwd()}")

        print(f"\n=== 成功: {filename} をGitHubリポジトリ {user_name}/{repo_name} に正常にアップロードしました ===")

    except Exception as e:
        print(f"予期せぬエラーが発生しました: {str(e)}")
        # スタックトレースを表示
        import traceback
        traceback.print_exc()

        # 元のディレクトリに戻る
        try:
            os.chdir(current_dir)
            print(f"元のディレクトリに戻りました: {os.getcwd()}")
        except:
            pass

# 実行
upload_to_github()

=== 処理開始: GitHubアップロード ===
設定情報: ユーザー名=KAFKA2306, リポジトリ名=hitaiall
アップロードファイル: index.html
ファイルサイズ: 33589 バイト
ファイル内容プレビュー: <!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content...

--- GitHubリポジトリ KAFKA2306/hitaiall にアップロード開始 ---
現在の作業ディレクトリ: /content

1. Gitの設定を行います...
Git設定完了

2. リポジトリをクローンします: /content/hitaiall
実行コマンド: git clone https://***@github.com/KAFKA2306/hitaiall.git /content/hitaiall
リポジトリのクローン成功

3. クローンされたリポジトリの内容を確認:

4. ファイルをリポジトリにコピーします:
コピー元: index.html
コピー先: /content/hitaiall/index.html
ファイルコピー成功
コピー後のファイルサイズ: 33589 バイト

5. GitHubにプッシュします...
現在のディレクトリ: /content
移動先ディレクトリ: /content/hitaiall
移動後のディレクトリ: /content/hitaiall

6. 変更を確認します (git status):

7. ファイルを追加します (git add):

8. 追加後の状態を確認します (git status):

9. 変更をコミットします (git commit):

10. 変更をプッシュします (git push):
元のディレクトリに戻りました: /content

=== 成功: index.html をGitHubリポジトリ KAFKA2306/hitaiall に正常にアップロードしました ===
